Simpsons Face Recognition Program
- Uses tensorflow and Keras 3
- Uses GPU

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [2]:
import os
import caer
import canaro
import numpy as np
import cv2 as cv
import gc

In [3]:
# model expects all image data to be of the same size
# i.e. we will have to resize all images in dataset to a particular size
IMG_SIZE = (80,80)
channels = 1 # i.e. grayscale
character_path = r'./simpsons_data/simpsons_dataset/' # path to dataset

In [ ]:
# Creating a character dictionary, sorting it in descending order

# We want to get top 10 characters with most images for that class
# - go through every folder, get name of folder and number of images
#   in the folder, and store this info in a dictionary (hashmap)
character_dict = {}
for character in os.listdir(character_path):
    character_dict[character] = len(os.listdir(os.path.join(character_path, character)))

# sort in decending order
character_dict = caer.sort_dict(character_dict, descending=True)

character_dict

In [ ]:
# Get the names of the top 10 people with most images
characters = []
count = 0
for i in character_dict:
    characters.append(i[0])
    count +=1
    if count >= 10:
        break

characters

In [ ]:
# Create the training data
# - go through every folder inside character_path, and look at every element
#   inside the characters list
# - it will the images from the top 10 people, to the training set
# - a training set is a list. Each element in the list was another list
#   of image array and its label
train = caer.preprocess_from_dir(character_path, characters, channels=channels, IMG_SIZE=IMG_SIZE, isShuffle=True)

In [ ]:
len(train) # number of images (training samples) in the training set

In [ ]:
# Visualizing the data (OpenCV doesn't display well in Jupyter notebooks)

# diplay an image in the training set
import matplotlib.pyplot as plt
plt.figure(figsize=(30,30))
plt.imshow(train[0][0], cmap='gray')
plt.show()

In [9]:
# separate training set into labels and features (separate lists)
# and also reshape the featureSet into a 4D tensor, so that it can be 
# fed into the model without any restrictions
featureSet, labels = caer.sep_train(train, IMG_SIZE=IMG_SIZE)

In [10]:
# from tensorflow.python.keras.utils.np_utils import to_categorical # NOTE: Needed utils.np_utils
from tensorflow.keras.utils import to_categorical

# Normalize the featureSet ==> make data within range of (0,1)
featureSet = caer.normalize(featureSet)

# convert numerical labels from class vector (integers) to binary class matrix.
labels = to_categorical(labels, len(characters))

In [11]:
# splitting featureset and labels into training set and validation set
# so in this case, 20% goes to validation, 80% goes to training
x_train, x_val, y_train, y_val = caer.train_val_split(featureSet, labels, val_ratio=.2)


# Extra stuff:
# import sklearn.model_selection as skm
# split_data = skm.train_test_split(featureSet, labels, test_size=.2)
# x_train, x_val, y_train, y_val = (np.array(item) for item in split_data)

In [ ]:
# can now delete unneeded variables
del train 
del featureSet
del labels
gc.collect()

In [13]:
# Useful variables when training
BATCH_SIZE = 32
EPOCHS = 10

In [14]:
# Image data generator
# - image generator that makes new images from existing ones to improve our network
# - introduces randomness in network ==> better accuracy
datagen = canaro.generators.imageDataGenerator()
train_gen = datagen.flow(x_train, y_train, batch_size=BATCH_SIZE)


In [ ]:
# Old code from video, does not work:
# model = canaro.models.createSimpsonsModel(IMG_SIZE=IMG_SIZE, channels=channels, 
#                                           output_dim=len(characters), loss='binary_crossentropy', decay=1e-6, learning_rate=0.001, momentum=0.9, nesterov=True)

# Works:
# Create our model (returns a compiled model)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D

output_dim=10

w, h = IMG_SIZE[:2]

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(w, h,channels)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(256, (3, 3), padding='same', activation='relu')) 
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(1024, activation='relu'))

# Output Layer
model.add(Dense(output_dim, activation='softmax'))

# from keras.models import Sequential, Model
# from keras.layers import Conv2D, Flatten, MaxPooling2D, Dense, Input, Reshape, Concatenate, GlobalAveragePooling2D, BatchNormalization, Dropout, Activation, GlobalMaxPooling2D
# from keras.utils import Sequence
# import tensorflow
# # Create our model (returns a compiled model)
# def create_ST_layer(input_shape = (64, 128, 3)):
#     input_img = Input(shape=input_shape)
#     model = Conv2D(48, kernel_size=(5, 5), input_shape = input_shape, strides = (1, 1), activation = 'relu')(input_img)
#     model = MaxPooling2D(pool_size=(2, 2), strides = (2, 2))(model)
#     model = Conv2D(32, kernel_size=(5, 5), strides = (1, 1), activation = 'relu')(model)
#     model = MaxPooling2D(pool_size=(2, 2), strides = (2, 2))(model)
#     model = Dense(50, activation = 'relu')(model)
#     model = Dense(6)(model)
#     model = tensorflow.keras.Model(inputs=input_img, outputs= model)
#     return model


In [ ]:
# model = create_ST_layer()
model.summary()

In [ ]:
# Training the model
model.compile(loss='binary_crossentropy')
# create a callbacks list
# contains a learning rate scheduler - schedules learning rate at certain intervals to allow the network to train better
from tensorflow.keras.callbacks import LearningRateScheduler
callbacks_list = [LearningRateScheduler(canaro.lr_schedule)]

training = model.fit(train_gen,
                    steps_per_epoch=len(x_train)//BATCH_SIZE,
                    epochs=EPOCHS,
                    validation_data=(x_val,y_val),
                    validation_steps=len(y_val)//BATCH_SIZE,
                    callbacks = callbacks_list)

TESTING

In [ ]:
test_path = r'./simpsons_data/simpsons_dataset/charles_montgomery_burns/pic_0011.jpg'

img = cv.imread(test_path)

plt.imshow(img)
plt.show()

In [20]:
def prepare(image):
    image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    image = cv.resize(image, IMG_SIZE)
    image = caer.reshape(image, IMG_SIZE, 1)
    return image

In [ ]:
predictions = model.predict(prepare(img))

In [ ]:
# Getting class with the highest probability
print(characters[np.argmax(predictions[0])])